In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Dataset paths
train_dir = 'C:/Users/SAYANDIP/Desktop/Interview dataset-Age Detection/Train'
test_dir = 'C:/Users/SAYANDIP/Desktop/Interview dataset-Age Detection/Test'
train_csv_file = 'C:/Users/SAYANDIP/Desktop/Interview dataset-Age Detection/train.csv'

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Set target image size and batch size
img_size = (64, 64)
batch_size = 32

In [7]:
train_csv = pd.read_csv(train_csv_file)

In [8]:
# Split data into training and validation sets
train_data = train_csv.sample(frac=0.8, random_state=42)
validation_data = train_csv.drop(train_data.index)


In [9]:
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    directory=train_dir,
    x_col='ID',
    y_col='Class',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 15917 validated image filenames belonging to 3 classes.


In [10]:
validation_generator = test_datagen.flow_from_dataframe(
    validation_data,
    directory=train_dir,
    x_col='ID',
    y_col='Class',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3979 validated image filenames belonging to 3 classes.


In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [12]:
# Compile model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
epochs = 10
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // batch_size)

Epoch 1/10
497/497 [==============================] - 51s 100ms/step - loss: 0.8883 - accuracy: 0.5856 - val_loss: 0.7629 - val_accuracy: 0.6739
Epoch 2/10
497/497 [==============================] - 35s 70ms/step - loss: 0.7982 - accuracy: 0.6504 - val_loss: 0.7524 - val_accuracy: 0.6845
Epoch 3/10
497/497 [==============================] - 35s 70ms/step - loss: 0.7563 - accuracy: 0.6698 - val_loss: 0.6833 - val_accuracy: 0.7054
Epoch 4/10
497/497 [==============================] - 35s 70ms/step - loss: 0.7171 - accuracy: 0.6908 - val_loss: 0.6540 - val_accuracy: 0.7266
Epoch 5/10
497/497 [==============================] - 35s 70ms/step - loss: 0.6882 - accuracy: 0.7048 - val_loss: 0.6409 - val_accuracy: 0.7331
Epoch 6/10
497/497 [==============================] - 34s 68ms/step - loss: 0.6639 - accuracy: 0.7150 - val_loss: 0.6357 - val_accuracy: 0.7321
Epoch 7/10
497/497 [==============================] - 34s 69ms/step - loss: 0.6442 - accuracy: 0.7267 - val_loss: 0.6163 - val_accuracy

In [15]:
# Evaluate model on validation data
validation_generator.reset()
loss, accuracy = model.evaluate(validation_generator)
print('Validation Loss:', loss)
print('Validation Accuracy:', accuracy)

125/125 [==============================] - 2s 16ms/step - loss: 0.5744 - accuracy: 0.7562
Validation Loss: 0.5744268298149109
Validation Accuracy: 0.7562201619148254


In [16]:
import os
import shutil

# Paths to test and train directories
test_dir = 'C:/Users/SAYANDIP/Desktop/Interview dataset-Age Detection/Test'
train_dir = 'C:/Users/SAYANDIP/Desktop/Interview dataset-Age Detection/Train'

# Path to CSV file
csv_file = 'C:/Users/SAYANDIP/Desktop/Interview dataset-Age Detection/train.csv'

# Create subdirectories in the test directory
with open(csv_file, 'r') as file:
    next(file)  # Skip header row
    for line in file:
        image_name, class_name = line.strip().split(',')
        class_dir = os.path.join(test_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        shutil.copy2(os.path.join(train_dir, image_name), class_dir)

In [18]:
test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 19896 images belonging to 3 classes.


In [19]:
# Generate predictions
test_data.reset()
y_pred = model.predict(test_data)
y_pred_labels = np.argmax(y_pred, axis=1)
class_labels = list(test_data.class_indices.keys())

622/622 [==============================] - 79s 127ms/step


In [20]:
test_labels = []
for i in range(len(test_data)):
    _, labels = test_data[i]
    test_labels.extend(np.argmax(labels, axis=1))
test_labels = np.array(test_labels)


In [31]:
report = classification_report(test_labels, y_pred_labels, target_names=class_labels)
print(report)

              precision    recall  f1-score   support

      MIDDLE       0.82      0.79      0.81     10796
         OLD       0.61      0.56      0.58      2396
       YOUNG       0.75      0.81      0.78      6704

    accuracy                           0.77     19896
   macro avg       0.73      0.72      0.72     19896
weighted avg       0.77      0.77      0.77     19896



In [30]:
# Print predictions for each image
for i in range(len(test_data.filenames)):
    image_path = test_data.filepaths[i]
    image_name = os.path.basename(image_path)
    prediction = class_labels[y_pred_labels[i]]
    print(f"Image: {image_name} | Prediction: {prediction}")

Image: 0.jpg | Prediction: MIDDLE
Image: 100.jpg | Prediction: MIDDLE
Image: 10000.jpg | Prediction: MIDDLE
Image: 10004.jpg | Prediction: MIDDLE
Image: 10016.jpg | Prediction: MIDDLE
Image: 10017.jpg | Prediction: MIDDLE
Image: 10018.jpg | Prediction: YOUNG
Image: 1002.jpg | Prediction: MIDDLE
Image: 10021.jpg | Prediction: MIDDLE
Image: 10023.jpg | Prediction: MIDDLE
Image: 10026.jpg | Prediction: MIDDLE
Image: 10027.jpg | Prediction: OLD
Image: 10028.jpg | Prediction: YOUNG
Image: 10030.jpg | Prediction: MIDDLE
Image: 10031.jpg | Prediction: MIDDLE
Image: 10034.jpg | Prediction: MIDDLE
Image: 10035.jpg | Prediction: YOUNG
Image: 1004.jpg | Prediction: OLD
Image: 10041.jpg | Prediction: MIDDLE
Image: 10045.jpg | Prediction: OLD
Image: 10046.jpg | Prediction: MIDDLE
Image: 10048.jpg | Prediction: MIDDLE
Image: 10053.jpg | Prediction: MIDDLE
Image: 10058.jpg | Prediction: MIDDLE
Image: 1006.jpg | Prediction: YOUNG
Image: 10065.jpg | Prediction: MIDDLE
Image: 1007.jpg | Prediction: MIDD

In [28]:

# Calculate accuracy
accuracy = np.mean(test_labels == y_pred_labels)
print('Accuracy:', accuracy)

Accuracy: 0.7717129071170085


In [29]:
accuracy_percentage = accuracy * 100
print('Accuracy:', accuracy_percentage, '%')

Accuracy: 77.17129071170085 %
